# 1.마스크 판별용 모델 학습

In [ ]:
# 디바이스 설정 (GPU 사용 가능하면 GPU 사용)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 정의
seg_model = smp.Unet(
    encoder_name='resnet50',        # 백본 네트워크 (원하는 대로 변경 가능)
    encoder_weights='imagenet',     # 사전 학습된 가중치 사용
    in_channels=1,                  # 입력 채널 수 (그레이스케일 이미지이므로 1)
    classes=1,                      # 출력 채널 수 (마스크이므로 1)
    activation=None,                # 후처리에서 시그모이드 적용 예정
)

seg_model.to(device)


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 398MB/s]


Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      

In [ ]:
# 손실 함수 정의 (이진 크로스 엔트로피 손실)
seg_criterion = nn.BCEWithLogitsLoss()

# 옵티마이저 정의
seg_optimizer = torch.optim.Adam(seg_model.parameters(), lr=1e-4)


In [ ]:
num_epochs = 3  # 필요에 따라 조정하세요.

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    seg_model.train()
    train_loss = 0

    for batch in tqdm(train_dataloader):
        images = batch['images_gray_masked'].to(device)  # 입력 이미지
        masks = batch['masks'].unsqueeze(1).float().to(device)  # 마스크 (채널 차원 추가 및 float 변환)

        # 순전파
        outputs = seg_model(images)
        loss = seg_criterion(outputs, masks)

        # 역전파 및 옵티마이저 스텝
        seg_optimizer.zero_grad()
        loss.backward()
        seg_optimizer.step()

        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_dataloader)
    print(f'Train Loss: {avg_train_loss:.4f}')



    # 모델 체크포인트 저장 (필요한 경우)
    #torch.save(seg_model.state_dict(), f'segmodel_epoch_{epoch+1}.pth')
    """Train Loss: 0.0371"""

    """100%
 5476/5476 [13:17<00:00,  6.54it/s]
Train Loss: 0.0265
Epoch 2/2
100%
 5476/5476 [13:17<00:00,  6.71it/s]
Train Loss: 0.0049"""


Epoch 1/3


  0%|          | 0/5476 [00:00<?, ?it/s]

Train Loss: 0.0383
Epoch 2/3


  0%|          | 0/5476 [00:00<?, ?it/s]

Train Loss: 0.0036
Epoch 3/3


  0%|          | 0/5476 [00:00<?, ?it/s]

Train Loss: 0.0025


In [ ]:
torch.save(seg_model.state_dict(), f'segmodel_epoch_{epoch+1}.pth')

# 2. 테스트 데이터셋에 적용

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import numpy as np
import segmentation_models_pytorch as smp
from tqdm import tqdm


In [ ]:
class TestImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_filenames = [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        # 이미지 파일 이름 가져오기
        image_filename = self.image_filenames[idx]
        image_path = os.path.join(self.image_dir, image_filename)

        # 이미지 로드 (그레이스케일)
        image = Image.open(image_path).convert('L')

        # 원본 이미지 크기 저장 (필요한 경우)
        original_size = image.size  # (Width, Height)

        # 필요한 경우 이미지 변환 적용

            # 기본적으로 텐서로 변환
        image = transforms.ToTensor()(image)

        # 딕셔너리로 반환 (이미지, 파일 이름, 원본 크기)
        return {
            'image': image,
            'filename': image_filename,
            'original_size': original_size
        }


In [ ]:

# 테스트 데이터셋 및 데이터로더 생성
test_image_dir = '/content/lama-with-refiner/extracted_files/test_input'
test_dataset = TestImageDataset(image_dir=test_image_dir)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
# 저장할 폴더 생성
output_dir = 'predicted_masks3'
os.makedirs(output_dir, exist_ok=True)

seg_model.eval()
with torch.no_grad():
    for batch in tqdm(test_dataloader):
        images = batch['image'].to(device)
        filenames = batch['filename']  # 파일 이름 리스트
        original_sizes = batch['original_size']  # 원본 이미지 크기 리스트

        # 모델 추론
        outputs = seg_model(images)

        # 시그모이드 함수 적용하여 확률로 변환
        probs = torch.sigmoid(outputs)

        # 이진화 (임계값 0.5 사용)
        preds = (probs > 0.5).float()

        # 마스크를 CPU로 이동하고 넘파이 배열로 변환
        mask_np = preds.squeeze().cpu().numpy() * 255  # [0, 255] 범위로 스케일링

        # 배열의 차원을 확인하고 2차원으로 변환
        if len(mask_np.shape) == 3:  # 만약 shape가 (C, H, W) 형태라면
            mask_np = mask_np[0]  # (C, H, W) -> (H, W)

        # 이미지 저장
        mask_image = Image.fromarray(mask_np.astype(np.uint8))

        # 원본 이미지 크기 확인
        original_size = original_sizes[0]  # 배치 크기가 1이므로 첫 번째 요소 사용

        # original_size가 tensor일 경우 처리
        if isinstance(original_size, torch.Tensor):
            if len(original_size) == 1:  # 단일 값인 경우
                original_size = (int(original_size.item()), int(original_size.item()))  # 정사각형 처리
            elif len(original_size) == 2:  # (H, W) 형태
                original_size = (int(original_size[1].item()), int(original_size[0].item()))  # (H, W) -> (W, H)
            else:
                raise ValueError(f"Unexpected original_size tensor format: {original_size}")

        # 원본 이미지 크기로 리사이즈
        mask_image = mask_image.resize(original_size, resample=Image.NEAREST)

        # 파일 이름 가져오기
        filename = filenames[0]  # 배치 크기가 1이므로 첫 번째 요소 사용

        # 마스크 파일 저장 경로 지정
        output_path = os.path.join(output_dir, filename)
        mask_image.save(output_path)


100%|██████████| 100/100 [00:01<00:00, 50.95it/s]
